Replication of the paper Image is Worth 16x16 Words: Transformers for Image Recognition at Scale in pytorch

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
from torchinfo import summary
from torch import nn
from PIL import Image
from torchvision import transforms , datasets
from torch.utils.data import DataLoader

In [ ]:
torch.__version__

In [ ]:
torchvision.__version__

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

## Dataset Prep

In [ ]:
train_dir = "../datasets/pizza_steak_sushi/train"
test_dir = "../datasets/pizza_steak_sushi/test"

In [ ]:
image = Image.open("../datasets/pizza_steak_sushi/train/pizza/5764.jpg")
print("image_size" , image.size)
image

In [ ]:
img_size = 224

transform = transforms.Compose([
    transforms.Resize((img_size , img_size)) , 
    transforms.ToTensor(),
])

In [ ]:
batch_size = 32

#create datasets
train_data = datasets.ImageFolder(
    "../datasets/pizza_steak_sushi/train" , 
    transform = transform
)

test_data = datasets.ImageFolder(
    "../datasets/pizza_steak_sushi/test" , 
    transform = transform
)

In [ ]:
train_dataloader = DataLoader(
    dataset = train_data , 
    batch_size = batch_size , 
    shuffle = True , 
    num_workers = 4 , 
    pin_memory= True
)

test_dataloader = DataLoader(
    dataset = test_data , 
    batch_size = batch_size , 
    shuffle = False , 
    num_workers = 4 , 
    pin_memory = True
)

train_dataloader , test_dataloader 

In [ ]:
classes = train_data.classes
classes

In [ ]:
# batch of images
image_batch , label_batch = next(iter(train_dataloader))
print("image_batch" , image_batch.shape)
print("label_batch" , label_batch.shape)
# get single image from batch
image , label = image_batch[0] , label_batch[0]

image.shape , label

In [ ]:
img = []
for image in image_batch:
    image = image.permute(1 , 2 , 0)
    img.append(image)
    
grid_size = int(np.ceil(np.sqrt(batch_size))) 
#np.ceil --> rounding
fig, axs = plt.subplots(grid_size , grid_size , figsize=(20, 20))

for i in range(grid_size):
    for j in range(grid_size):
        ax = axs[i , j ]
        if i * grid_size +j <batch_size:
            ax.imshow(img[i * grid_size+ j], cmap='gray' , aspect='auto')
            ax.axis('off')
        else:
            ax.axis('off')
plt.subplots_adjust(wspace = 0.1  , hspace= 0.1)
plt.show()

## Model

### Patch Embedding

In [ ]:
# images are 224 x 224 x 3
height = 224 # H
width = 224 # W
channels = 3 # C
patch_size = 16 # P

numb_patches = int((height * width) / patch_size**2)
print(f"Number of Patches (N) with image resolution {height}x{width} is {numb_patches} patches")

In [ ]:
embedding_layer_input_shape = (height , width , channels)

# 196 patches each of size 16 x 16
# output will be for each image --> 192 patches , P**2 * C
embedding_layer_output_shape = (numb_patches , patch_size**2 * channels) 

print(f"Input Image shape : {embedding_layer_input_shape}")
print(f"Output embedded shape flattened to patches : {embedding_layer_output_shape}")

In [ ]:
image , label = image_batch[5] , label_batch[5]
plt.imshow(image.permute(1,2,0))
plt.title(classes[label])
plt.axis("off")
plt.show()

In [ ]:
# visualizing top row patched pixels

permuted_image = image.permute(1,2,0) # H W C

plt.figure(figsize=(patch_size , patch_size))
plt.imshow(permuted_image[:patch_size , : , :])

In [ ]:
img_size = 224 
patch_size = 16
num_patches = img_size / patch_size

assert img_size % patch_size ==0 , "image size must be divisible by patch_size"
print(f"number of patches per row: {num_patches}\nPatch size is {patch_size} x {patch_size} pixels")

fig,axs = plt.subplots(nrows = 1, 
                       ncols = img_size // patch_size ,
                       figsize = (num_patches , num_patches) , 
                       sharex = True , 
                       sharey = True)

for i,patch in enumerate(range(0 , img_size , patch_size)): #start from zero move by patch size in img_size
    axs[i].imshow(permuted_image[:patch_size , patch:patch+patch_size , :])
    axs[i].set_xlabel(i+1)
    axs[i].set_xticks([])
    axs[i].set_yticks([])
    

In [ ]:
img_size = 224
patch_size = 16
num_patches = img_size / patch_size

assert img_size % patch_size  == 0 , "the img size should be divisible by patch size"
print(f"Number of patches per row: {num_patches}\
        \nNumber of patches per column: {num_patches}\
        \nTotal patches: {num_patches*num_patches}\
        \nPatch size: {patch_size} pixels x {patch_size} pixels")

fig , axs = plt.subplots(
    nrows = img_size // patch_size , 
    ncols = img_size // patch_size , 
    figsize = (num_patches , num_patches),
    sharex = True , 
    sharey = True
)

for i , patch_height in enumerate(range(0 , img_size ,patch_size )):
   for j , patch_width in enumerate(range(0 , img_size , patch_size)):
       
    axs[i , j].imshow(permuted_image[patch_height:patch_height+patch_size , 
                                        patch_width:patch_width+patch_size , :])
    axs[i, j].set_ylabel(i+1,
                             rotation="horizontal",
                             horizontalalignment="right",
                             verticalalignment="center")
    axs[i, j].set_xlabel(j+1)
    axs[i, j].set_xticks([])
    axs[i, j].set_yticks([])
    axs[i, j].label_outer()
        
fig.suptitle(f"{classes[label]} -> Patchified", fontsize=16)
plt.show()

In [ ]:
patch_size = 16
embed_dim = 768 #D: number of feature / activation maps

conv2d = nn.Conv2d(
    in_channels = 3 , 
    out_channels = embed_dim , 
    kernel_size = patch_size , 
    stride = patch_size , 
    padding = 0
)

In [ ]:
plt.imshow(image.permute(1, 2, 0))
plt.title(classes[label])
plt.axis("off")

In [ ]:
print("image shape before: " , image.shape)
# expected shape for the Conv2d is (N , C , H , W)
image = image.unsqueeze(0) 
print("image shape after" , image.shape)

img_conv = conv2d(image)
print("image shape after Conv2D : " , img_conv.shape)
# output shape [batch_size , embed_dim , feature_map_height , feature_map_width]

In [ ]:
import random 

random_indices = random.sample(range(0 , 758) , k=10)

print(f"showing random convolutional feature maps from indices : {random_indices}" )

fig , axs = plt.subplots(nrows = 1 , ncols=len(random_indices) , figsize=(12,12))

for i,idx in enumerate(random_indices):
    image_conv_feat_map = img_conv[: , idx , : , :]
    
    axs[i].imshow(image_conv_feat_map.squeeze().detach().numpy())
    # detach takes a copy of the tensor that disconnected from the gradient graph 
    axs[i].set(xticklabels = [] , yticklabels = [] , xticks = [] ,
               yticks = []) 

In [ ]:
# After turning the image into patch embedding , its time to flatten it

print(f"shape of the output of the conv : {img_conv.shape} -> [batch , embedding_dim , feature_map_height, feature_map_width]")
# what we want to flatten is the spatial dimension of the feature map

flatten = nn.Flatten()
t = flatten(img_conv)
print(t.shape) # this is flattening the whole tensor (1 , 768 * 14 * 14)

flatten = nn.Flatten(start_dim= 2 , end_dim = 3 )
t = flatten(img_conv)

print(t.shape)

In [ ]:
image , label = image_batch[5] , label_batch[5]

plt.imshow(image.permute(1 , 2, 0))
plt.title(classes[label])
plt.axis(False)
print(f"original image shape: {image.shape}")

image_conv = conv2d(image.unsqueeze(0)) # (N , C, H , W)
print(f"shape after convolution: {image_conv.shape}")# 768 feature maps each of size 14 x 14

image_flatten = flatten(image_conv)
print(f"shape after flatten: {image_flatten.shape}") # 768 feature maps each of flattend size of 196

print("desired shape : N x(P^2 * C) --> (196 , 768)")

image_final = image_flatten.permute(0 , 2 , 1)
print(f"Patch embedding final shape : {image_final.shape} ")

In [ ]:
single_sample = image_final[: , : , 0]
print("shape of this sample : " , single_sample.shape) # 2D image to 1D embedding vector
plt.figure(figsize=(20 , 20))
plt.imshow(single_sample.detach().numpy())
plt.title(f"Flattened feature map shape: {image_final.shape}")
plt.axis("off")

Summing up everything till now in a module

In [ ]:
class PatchEmbedding(nn.Module):
    """
    turns 2d input image into a 1D sequence learnable embedding vector
    
    Args:
    in_channels(int) : Number of channels in the input image. (Default set to 3)
    patch_size (int) : size of the patch (Defaults to 16)
    embed_dim (int) : number of feature maps (Defaults to 768) 
    """
    def __init__(self ,
                 in_channels: int = 3 ,
                 patch_size : int = 16 , 
                 embed_dim: int = 768 ):
        super().__init__()
        self.conv2d = nn.Conv2d(in_channels= in_channels , 
                           out_channels = embed_dim , 
                           kernel_size= patch_size , 
                           stride = patch_size , 
                           padding = 0)
        
        self.flatten = nn.Flatten(start_dim = 2 , end_dim = 3)
    
    def forward(self , image):
        height = image.shape[-1]
        assert height % patch_size == 0, f"Input image size must be divisble by patch size, image shape: {height}, patch size: {patch_size}"
        patched_image = self.conv2d(image)
        flattened_image = self.flatten(patched_image)
        
        return flattened_image.permute(0 , 2 , 1)

In [ ]:
patchify = PatchEmbedding(in_channels=3,
                          patch_size=16,
                          embed_dim=768)
# Pass a single image through
print(f"Input image shape: {image.unsqueeze(0).shape}")
patch_embedded_image = patchify(image.unsqueeze(0)) # add an extra batch dimension on the 0th index, otherwise will give error
print(f"Output patch embedding shape: {patch_embedded_image.shape}")

In [ ]:
# Create random input sizes
random_input_image = (1, 3, 224, 224)
random_input_image_error = (1, 3, 250, 250) # will error because image size is incompatible with patch_size

# Get a summary of the input and outputs of PatchEmbedding (uncomment for full output)
summary(PatchEmbedding(),
        input_size=random_input_image, # try swapping this for "random_input_image_error"
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
print("patch_embedded_image shape " , patch_embedded_image.shape)

batch_size , embedding_dim = patch_embedded_image.shape[0] , patch_embedded_image.shape[-1]

class_token = nn.Parameter(torch.ones((batch_size , 1 , embedding_dim)) ,
                           requires_grad= True)# [batch_size, number_of_tokens, embedding_dimension]

print(f"class_token: {class_token[: , : , :10]}")
print(f"class_token shape: {class_token.shape}")

In [ ]:
updated_patch = torch.cat((class_token , patch_embedded_image) , 
                          dim = 1)

print(f"shape of updated patch embedding with class token: {updated_patch.shape}")

# before each feature map was represented with 196 tokens  , however now after adding the class token each will be represented by 197

print(updated_patch)

## Position Embedding

Epos --> (N+1 X D) where N is the number of patches and the 1 is for class token

In [ ]:
updated_patch , updated_patch.shape


In [ ]:
numb_patches = int((height * width) / patch_size **2)

embedding_dim = updated_patch.shape[2]

position_embedding = nn.Parameter(
    torch.ones( 1 , numb_patches +1 , embedding_dim),
                        requires_grad= True        ) # learnable

position_embedding[: , :10 , :10] , position_embedding.shape


In [ ]:
patch_with_position = updated_patch + position_embedding
patch_with_position , patch_with_position.shape

In [ ]:
#full code 

patch_size = 16

image , label = image_batch[6] , label_batch[6]

print(f"image tensor shape : {image.shape}")
height , width = image.shape[1] , image.shape[2]

x = image.unsqueeze(0) # add batch dim
print(f"Input image with batch dimension shape: {x.shape}")

patch_embedding_layer = PatchEmbedding(in_channels = image.shape[0] ,
                                       patch_size = patch_size , 
                                       embed_dim = 768)
patch_embedding = patch_embedding_layer(x)
print(f"Patching embedding shape: {patch_embedding.shape}")

batch_size = patch_embedding.shape[0]
embedding_dimension = patch_embedding.shape[-1]
class_token = nn.Parameter(torch.ones(batch_size , 1 , embedding_dimension),
                           requires_grad= True)
print(f"Class token embedding shape: {class_token.shape}")

patch_embedding_with_class = torch.cat((class_token , patch_embedding) , dim = 1)
print(f"Patch embedding with class token shape: {patch_embedding_with_class.shape}")

# position embedding

number_of_patches = int((height*width) / patch_size**2)

position_embedding = nn.Parameter(torch.ones(1 , number_of_patches + 1 , embedding_dim),
                                  requires_grad=True)

patch_and_position_embedd = patch_embedding_with_class + position_embedding
print(f"Patch and position embedding shape: {patch_and_position_embedd.shape}")


Multi-Head Attention

In [ ]:
#transformer encoder: 

class MultiheadSelfAttentionBlock(nn.Module):
    def __init__(self, 
                 embedding_dim: int= 768 , 
                 n_heads: int=12 , 
                 attn_dropout: float = 0):
        super().__init__()
        
        self.layer_norm = nn.LayerNorm(normalized_shape = embedding_dim)
        
        self.multihead_attn = nn.MultiheadAttention(
            embed_dim = embedding_dim , 
            num_heads = n_heads , 
            dropout = attn_dropout , 
            batch_first = True
        )
        
    def forward(self , x):
        
        x = self.layer_norm(x)
        attn_output , _ = self.multihead_attn(query = x , 
                                              value = x , 
                                              key = x , 
                                              need_weights = False)
        
        return attn_output

In [ ]:
multihead_self_attention_block = MultiheadSelfAttentionBlock(
    embedding_dim = 768 , 
    n_heads= 12
)

patched_image_through_msa_block = multihead_self_attention_block(patch_and_position_embedd)
print(f"Input shape of MSA block: {patch_and_position_embedd.shape}")
print(f"Output shape MSA block: {patched_image_through_msa_block.shape}")

Multilayer Perceptron(MLP)

In [ ]:
# layer norm -> linear layer -> non-linear layer -> dropout -> linear layer -> dropout

class MLPBlock(nn.Module):
    def __init__(self, 
                 embedding_dim: int=768 , 
                 mlp_size: int = 3072 , # from paper
                 dropout: float = 0.1): #from paper
        super().__init__()
        
        self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)
        
        self.mlp = nn.Sequential(
            nn.Linear(in_features = embedding_dim , 
                      out_features = mlp_size) , 
            nn.GELU() , 
            nn.Dropout(p = dropout) , 
            nn.Linear(in_features = mlp_size , 
                      out_features = embedding_dim) , 
            nn.Dropout(p = dropout)
        )
        
    def forward(self , x):
        x = self.layer_norm(x)
        mlp = self.mlp(x)
        
        return x

In [ ]:
mlp_block = MLPBlock(embedding_dim = 768 , 
                     mlp_size = 3072 , 
                     dropout = 0.1)

mlp_out = mlp_block(patched_image_through_msa_block)
print(f"Input shape of MLP block: {patched_image_through_msa_block.shape}")
print(f"Output shape MLP block: {mlp_out.shape}")

In [ ]:
#TRANSFORMER ENCODER : 
# x_input -> MSA_block -> [MSA_block_output + x_input] -> MLP_block -> [MLP_block_output + MSA_block_output + x_input] -> ...

class TransformerEncoderBlock(nn.Module):
    def __init__(self , 
                 embedding_dim: int = 768 , 
                 num_heads: int = 12 , 
                 mlp_size : int = 3072 , 
                 mlp_dropout : float = 0.1 , 
                 attn_dropout : float = 0):
        super().__init__()
        
        self.msa_block = MultiheadSelfAttentionBlock(
            embedding_dim= embedding_dim , 
            n_heads = num_heads , 
            attn_dropout= attn_dropout
        )
        self.mlp_block = MLPBlock(
            embedding_dim= 768 , 
            mlp_size = mlp_size , 
            dropout = mlp_dropout
        )
        
    def forward(self , x):
        
        msa = self.msa_block(x) + x
        
        mlp = self.mlp_block(msa) + msa 
        
        return mlp

In [ ]:


# Create an instance of TransformerEncoderBlock
transformer_encoder_block = TransformerEncoderBlock()

# Print an input and output summary of our Transformer Encoder (uncomment for full output)
summary(model=transformer_encoder_block,
        input_size=(1, 197, 768), # (batch_size, num_patches, embedding_dimension)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])


In [ ]:
#transformer encoder with pytorch

torch_transformer_encoder_layer = nn.TransformerEncoderLayer(
    d_model = 768 , 
    nhead = 12 , 
    dim_feedforward = 3072 , 
    dropout = 0.1 , 
    activation= 'gelu' , 
    batch_first= True , 
    norm_first = True
)

torch_transformer_encoder_layer

In [ ]:
# Get the output of PyTorch's version of the Transformer Encoder (uncomment for full output)
summary(model=torch_transformer_encoder_layer,
        input_size=(1, 197, 768), # (batch_size, num_patches, embedding_dimension)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])


FULL ViT code:

In [ ]:
class Vit(nn.Module):
    def __init__(self, 
                 img_size:int=224 ,
                 in_channels:int=3 , 
                 embedding_dim:int=768,
                 patch_size:int=16 , 
                 num_transformer_layers:int=12,
                 mlp_size:int=3072,
                 num_heads:int=12,
                 attn_dropout:float=0,
                 mlp_dropout:float=0.1 , 
                 embedding_dropout:float=0.1 , 
                 num_classes:int=1000):
        super().__init__()
        assert img_size % patch_size == 0, f"Image size must be divisible by patch size, image size: {img_size}, patch size: {patch_size}."
        self.num_patches = (img_size * img_size) // patch_size**2
        self.class_embed = nn.Parameter(data = torch.randn(1 , 1 , embedding_dim),
                                        requires_grad=True)
        self.positional_embed = nn.Parameter(data=torch.randn(1, self.num_patches+1, embedding_dim),
                                               requires_grad=True)
        self.embedding_dropout = nn.Dropout(p=embedding_dropout)
        self.patch_embedding = PatchEmbedding(in_channels = in_channels , 
                                              patch_size = patch_size , 
                                              embed_dim = embedding_dim)
        self.transformer_encoder = nn.Sequential(*[TransformerEncoderBlock(
            embedding_dim= embedding_dim , 
            num_heads= num_heads , 
            mlp_size= mlp_size , 
            mlp_dropout= mlp_dropout
        )for _ in range(num_transformer_layers)])
        
        self.classifier = nn.Sequential(
            nn.LayerNorm(normalized_shape=embedding_dim),
            nn.Linear(in_features=embedding_dim , 
                      out_features=num_classes)
        )
        
        
    def forward(self , x):
        
        batch_size = x.shape[0]
        class_token = self.class_embed.expand(batch_size, -1, -1)
        
        patches = self.patch_embedding(x)
        
        patches_with_cls = torch.cat((class_token , patches) , dim =1)
        
        patch_with_position = patches_with_cls + self.positional_embed
        
        embed_drop = self.embedding_dropout(patch_with_position)
        
        transformer = self.transformer_encoder(embed_drop)
        
        classes = self.classifier(transformer[: , 0])
        
        return classes

In [ ]:
# Example of creating the class embedding and expanding over a batch dimension
batch_size = 32
class_token_embedding_single = nn.Parameter(data=torch.randn(1, 1, 768)) # create a single learnable class token
class_token_embedding_expanded = class_token_embedding_single.expand(batch_size, -1, -1) # expand the single learnable class token across the batch dimension, "-1" means to "infer the dimension"

# Print out the change in shapes
print(f"Shape of class token embedding single: {class_token_embedding_single.shape}")
print(f"Shape of class token embedding expanded: {class_token_embedding_expanded.shape}")

In [ ]:

# Create a random tensor with same shape as a single image
random_image_tensor = torch.randn(1, 3, 224, 224) # (batch_size, color_channels, height, width)

# Create an instance of ViT with the number of classes we're working with (pizza, steak, sushi)
vit = Vit(num_classes=len(classes))

# Pass the random image tensor to our ViT instance
vit(random_image_tensor)

In [ ]:
summary(model=vit,
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Training Code

In [ ]:
from engine import train

vit = Vit()
optimizer = torch.optim.Adam(
    params = vit.parameters() , 
    lr = 3e-3 , 
    betas = (0.9 , 0.999) , 
    weight_decay = 0.3

)

loss_fn = torch.nn.CrossEntropyLoss()

results = train(model = vit , 
                train_dataloader= train_dataloader , 
                test_dataloader = test_dataloader , 
                optimizer = optimizer , 
                loss_fn= loss_fn , 
                epochs = 10 , 
                device = device)

In [ ]:
loss = results['train_loss']
test_loss = results['test_loss']

accuracy = results['train_acc']
test_acc = results['test_acc']

epochs = range(len(results['train_loss']))

plt.figure(figsize=(15 , 7))

plt.subplot(1 , 2 , 1)
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, label="train_loss")
plt.plot(epochs, test_loss, label="test_loss")
plt.title("Loss")
plt.xlabel("Epochs")
plt.legend()

plt.subplot(1 ,  2 , 2)
plt.plot(epochs, accuracy, label="train_accuracy")
plt.plot(epochs, test_acc, label="test_accuracy")
plt.title("Accuracy")
plt.xlabel("Epochs")
plt.legend()

Using a pre-trained ViT from pytorch library

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT #default --> best available

pretrained_vit = torchvision.models.vit_b_16(weights = pretrained_vit_weights , progress=True).to(device)

for parameter in pretrained_vit.parameters():
    parameter.requires_grad = False
    
    
pretrained_vit.heads = nn.Linear(in_features= 768 , out_features=len(classes)).to(device)


In [ ]:
# Print a summary using torchinfo (uncomment for actual output)
summary(model=pretrained_vit,
        input_size=(32, 3, 224, 224), # (batch_size, color_channels, height, width)
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)